# Análise Exploratória de Dados: IBGE

<h1>
    <p align="center">
        Work in Progress 🛠️
    </p>
</h1>

Objetivos:

- Realizar uma análise focada nas variáveis referentes a população brasileira e alguns índices econômicos;

Algumas possibilidades: 

- Comparativos entre estados ou regiões do Brasil no quesito populacional e/ou econômico;
- Procurar entender quais os estados com os municípios mais ricos, mais populosos, com mais imigrantes;
- Rankeamento desses estados com base em alguns KPI's (GDP, IDH, População total);
- Um dado interessante para a análise também pode ser a distribuição de carros/motos por cidadão.
- Divisão etária da população desses estados, buscando quais os de população mais jovem e velha na média;

## Puxando dados e selecionando variáveis necessárias

In [173]:
# Importando pacotes:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Definindo parâmetros de tamanho das figuras do Seaborn:
sns.set(rc={'figure.figsize':(12,8)})
sns.set(style="darkgrid")

print('Pacotes prontos!')

Pacotes prontos!


In [3]:
# Criando dataframe:
data = pd.read_csv('data/BRAZIL_CITIES.csv', sep=';')

In [4]:
# Visualizando dados:
data.head()

,CITY,STATE,CAPITAL,IBGE_RES_POP,IBGE_RES_POP_BRAS,IBGE_RES_POP_ESTR,IBGE_DU,IBGE_DU_URBAN,IBGE_DU_RURAL,IBGE_POP,...,Pu_Bank,Pr_Assets,Pu_Assets,Cars,Motorcycles,Wheeled_tractor,UBER,MAC,WAL-MART,POST_OFFICES
0,Abadia De Goiás,GO,0,6876.0,6876.0,0.0,2137.0,1546.0,591.0,5300.0,...,NaN,NaN,NaN,2158.0,1246.0,0.0,NaN,NaN,NaN,1.0
1,Abadia Dos Dourados,MG,0,6704.0,6704.0,0.0,2328.0,1481.0,847.0,4154.0,...,NaN,NaN,NaN,2227.0,1142.0,0.0,NaN,NaN,NaN,1.0
2,Abadiânia,GO,0,15757.0,15609.0,148.0,4655.0,3233.0,1422.0,10656.0,...,1.0,33724584.0,67091904.0,2838.0,1426.0,0.0,NaN,NaN,NaN,3.0
3,Abaeté,MG,0,22690.0,22690.0,0.0,7694.0,6667.0,1027.0,18464.0,...,2.0,44974716.0,371922572.0,6928.0,2953.0,0.0,NaN,NaN,NaN,4.0
4,Abaetetuba,PA,0,141100.0,141040.0,60.0,31061.0,19057.0,12004.0,82956.0,...,4.0,76181384.0,800078483.0,5277.0,25661.0,0.0,NaN,NaN,NaN,2.0


In [5]:
# Checando shape:
data.shape

(5573, 81)

Dados: 5573 Registros em 81 Variáveis

In [6]:
# Checando colunas:
data.columns

Index(['CITY', 'STATE', 'CAPITAL', 'IBGE_RES_POP', 'IBGE_RES_POP_BRAS',
       'IBGE_RES_POP_ESTR', 'IBGE_DU', 'IBGE_DU_URBAN', 'IBGE_DU_RURAL',
       'IBGE_POP', 'IBGE_1', 'IBGE_1-4', 'IBGE_5-9', 'IBGE_10-14',
       'IBGE_15-59', 'IBGE_60+', 'IBGE_PLANTED_AREA', 'IBGE_CROP_PRODUCTION_$',
       'IDHM Ranking 2010', 'IDHM', 'IDHM_Renda', 'IDHM_Longevidade',
       'IDHM_Educacao', 'LONG', 'LAT', 'ALT', 'PAY_TV', 'FIXED_PHONES', 'AREA',
       'REGIAO_TUR', 'CATEGORIA_TUR', 'ESTIMATED_POP', 'RURAL_URBAN',
       'GVA_AGROPEC', 'GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC',
       ' GVA_TOTAL ', 'TAXES', 'GDP', 'POP_GDP', 'GDP_CAPITA', 'GVA_MAIN',
       'MUN_EXPENDIT', 'COMP_TOT', 'COMP_A', 'COMP_B', 'COMP_C', 'COMP_D',
       'COMP_E', 'COMP_F', 'COMP_G', 'COMP_H', 'COMP_I', 'COMP_J', 'COMP_K',
       'COMP_L', 'COMP_M', 'COMP_N', 'COMP_O', 'COMP_P', 'COMP_Q', 'COMP_R',
       'COMP_S', 'COMP_T', 'COMP_U', 'HOTELS', 'BEDS', 'Pr_Agencies',
       'Pu_Agencies', 'Pr_Bank', 'Pu_Bank', 'P

Antes de nos aprofundarmos mais no Dataset, iremos realizar um corte de variáveis, preservando apenas aquelas que interessam para a análise proposta no início.

In [61]:
# Selecionando as variáveis que serão relevantes para a análise:
data_pop = data.loc[
    :,
    ['CITY', 'STATE', 'CAPITAL', 'IBGE_RES_POP', 'IBGE_RES_POP_BRAS', 'IBGE_RES_POP_ESTR', 'IBGE_DU', 
    'IBGE_DU_URBAN', 'IBGE_DU_RURAL', 'IBGE_POP', 'IBGE_1', 'IBGE_1-4', 'IBGE_5-9', 'IBGE_10-14',
    'IBGE_15-59', 'IBGE_60+', 'Cars', 'Motorcycles', 'IDHM', 'IDHM_Renda', 
    'IDHM_Longevidade', 'IDHM_Educacao', 'TAXES', 'LAT', 'LONG']
].copy()


In [62]:
# Visualizando novos dados:
data_pop.head()

,CITY,STATE,CAPITAL,IBGE_RES_POP,IBGE_RES_POP_BRAS,IBGE_RES_POP_ESTR,IBGE_DU,IBGE_DU_URBAN,IBGE_DU_RURAL,IBGE_POP,...,IBGE_60+,Cars,Motorcycles,IDHM,IDHM_Renda,IDHM_Longevidade,IDHM_Educacao,TAXES,LAT,LONG
0,Abadia De Goiás,GO,0,6876.0,6876.0,0.0,2137.0,1546.0,591.0,5300.0,...,416.0,2158.0,1246.0,0.708,0.687,0.830,0.622,20554.20,-16.758812,-49.440548
1,Abadia Dos Dourados,MG,0,6704.0,6704.0,0.0,2328.0,1481.0,847.0,4154.0,...,589.0,2227.0,1142.0,0.690,0.693,0.839,0.563,12873.50,-18.487565,-47.396832
2,Abadiânia,GO,0,15757.0,15609.0,148.0,4655.0,3233.0,1422.0,10656.0,...,990.0,2838.0,1426.0,0.690,0.671,0.841,0.579,26822.58,-16.182672,-48.718812
3,Abaeté,MG,0,22690.0,22690.0,0.0,7694.0,6667.0,1027.0,18464.0,...,2681.0,6928.0,2953.0,0.698,0.720,0.848,0.556,26994.09,-19.155848,-45.446191
4,Abaetetuba,PA,0,141100.0,141040.0,60.0,31061.0,19057.0,12004.0,82956.0,...,5996.0,5277.0,25661.0,0.628,0.579,0.798,0.537,95180.48,-1.723470,-48.884404


Com nossas variáveis selecionadas, podemos dar início propriamente as análises iniciais do Dataset e limpeza.

## Análise inicial e conhecendo os dados 

In [63]:
# Visualizando o novo shape:
data_pop.shape

(5573, 25)

In [64]:
data_pop.columns

Index(['CITY', 'STATE', 'CAPITAL', 'IBGE_RES_POP', 'IBGE_RES_POP_BRAS',
       'IBGE_RES_POP_ESTR', 'IBGE_DU', 'IBGE_DU_URBAN', 'IBGE_DU_RURAL',
       'IBGE_POP', 'IBGE_1', 'IBGE_1-4', 'IBGE_5-9', 'IBGE_10-14',
       'IBGE_15-59', 'IBGE_60+', 'Cars', 'Motorcycles', 'IDHM', 'IDHM_Renda',
       'IDHM_Longevidade', 'IDHM_Educacao', 'TAXES', 'LAT', 'LONG'],
      dtype='object')

Agora temos os mesmos 5573 registros em apenas 23 variáveis

Para termos uma visão melhor e mais apropiada do conteúdo de cada variável, realizaremos um dicionário de variáveis, baseado no arquivo CSV presente na pasta `data` chamado `Data_Dictionary.csv`:


| <p align='center'>**Variável**</p>| <p align='center'>**Conteúdo**</p>                                                         |
|-----------------------------------|--------------------------------------------------------------------------------------------|
| *`CITY`*                          | Nome da cidade.                                                                            |
| *`STATE`*                         | Nome do estado.                                                                            |
| *`CAPITAL`*                       | Indica se a cidade nomeada no registro é a capital do estado.                              |
| *`IBGE_RES_POP`*                  | Número de habitantes da cidade de acordo com o senso de 2010.                              |
| *`IBGE_RES_POP_BRAS`*             | Quantos cidadãos da população total da cidade são brasileiros.                             |
| *`IBGE_RES_POP_ESTR`*             | Quantos cidadãos da população total da cidade são estrangeiros.                            |  
| *`IBGE_DU`*                       | Número de unidades domésticas totais.                                                      |
| *`IBGE_DU_URBAN`*                 | Das unidades domésticas totais, quantas estão em regiões urbanas.                          |
| *`IBGE_DU_RURAL`*                 | Das unidades domésticas totais, quantas estão em regiões rurais.                           |
| *`IBGE_POP`*                      | Moradores em domicílios particulares permanentes, em áreas com ordenamento urbano regular. |
| *`IBGE_1`*                        | Número de pessoas com até 1 ano de idade na população referente a tabela `IBGE_POP`.       |
| *`IBGE_1-4`*                      | Número de pessoas entre 1 e 4 anos de idade na população referente a tabela `IBGE_POP`.    |
| *`IBGE_5-9`*                      | Número de pessoas entre 5 e 9 anos de idade na população referente a tabela `IBGE_POP`.    |
| *`IBGE_10-14`*                    | Número de pessoas entre 10 e 14 anos de idade na população referente a tabela `IBGE_POP`.  |
| *`IBGE_15-59`*                    | Número de pessoas entre 15 e 59 anos de idade na população referente a tabela `IBGE_POP`.  |
| *`IBGE_60+`*                      | Número de pessoas a partir dos 60 anos de idade na população referente a tabela `IBGE_POP`.|
| *`Cars`*                          | Número total de carros referente ao ano de 2019.                                           |
| *`Motorcycles`*                   | Número total de Motocicletas referente ao ano de 2019.                                     |
| *`IDHM`*                          | Índice de Desenvolvimento Humano.                                                          |
| *`IDHM_Renda`*                    | Índice de Desenvolvimento Humano - Renda.                                                  |
| *`IDHM_Longevidade`*              | Índice de Desenvolvimento Humano - Longevidade.                                            |
| *`IDHM_Educacao`*                 | Índice de Desenvolvimento Humano - Educação                                                |
| *`TAXES`*                         | Montante total pago em Impostos.                                                           |
| *`LONG`*                          | Coordenada longitudinal da cidade.                                                         |
| *`LAT`*                           | Coordenada latitudinal da cidade.                                                          |

Os nomes das variáveis serão alterados mais adiante, para facilitar a compreensão das mesmas sem a necessidade de referenciar a tabela a todo momento. Este processo também sera realizado para padronizar a nomenclatura da melhor maneira possível.

In [65]:
# Checando informações específicas sobre as variáveis:
data_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5573 entries, 0 to 5572
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CITY               5573 non-null   object 
 1   STATE              5573 non-null   object 
 2   CAPITAL            5573 non-null   int64  
 3   IBGE_RES_POP       5565 non-null   float64
 4   IBGE_RES_POP_BRAS  5565 non-null   float64
 5   IBGE_RES_POP_ESTR  5565 non-null   float64
 6   IBGE_DU            5563 non-null   float64
 7   IBGE_DU_URBAN      5563 non-null   float64
 8   IBGE_DU_RURAL      5492 non-null   float64
 9   IBGE_POP           5565 non-null   float64
 10  IBGE_1             5565 non-null   float64
 11  IBGE_1-4           5565 non-null   float64
 12  IBGE_5-9           5565 non-null   float64
 13  IBGE_10-14         5565 non-null   float64
 14  IBGE_15-59         5565 non-null   float64
 15  IBGE_60+           5565 non-null   float64
 16  Cars               5562 

Algumas informações relevantes:

- No total, temos 22 variáveis do tipo float64, 1 do tipo int64 e 2 do tipo object. Visando a melhor performance possível do Dataset, iremos avaliar a possibilidade de alterar os Data Types para opções melhores (int8 ou 32 por exemplo, categorical para as variáveis object, etc.);

- Temos poucos dados faltantes espalhados em diversas colunas.

Dado o fato de que praticamente todas as colunas estão em float por mais que muitas não precisem estar, a avaliação estatística utilizando o método .describe() fica prejudicada, assim como a procura por outliers utilizando boxplots. Sendo assim, antes de continuar essas análises estatísticas, iremos realizar a retirada de valores nulos e as transformações de Data Types nos dados.

## Retirada de Nulls, Alterando Dtypes e Renomenado Colunas

In [177]:
# Criando cópia dos dados para a limpeza:
data_pop_clean = data_pop.copy()

- Renomenado Colunas:

In [178]:
# Renomeando e padronizando colunas:
data_pop_clean = data_pop_clean.rename(columns={
    'CITY': 'City', 'STATE': 'State', 'CAPITAL': 'Capital',
    'IBGE_RES_POP': 'Total_Pop', 'IBGE_RES_POP_BRAS': 'Total_Pop_Brazilians',
    'IBGE_RES_POP_ESTR': 'Total_Pop_Foreigners', 'IBGE_DU': 'Total_Uni_Dom',
    'IBGE_DU_URBAN': 'Total_Uni_Dom_Urb', 'IBGE_DU_RURAL': 'Total_Uni_Dom_Rur',
    'IBGE_POP': 'Total_Pop_Urb', 'IBGE_1': 'Pop_Urb_1', 'IBGE_1-4': 'Pop_Urb_1-4',
    'IBGE_5-9': 'Pop_Urb_5-9', 'IBGE_10-14': 'Pop_Urb_10-14', 'IBGE_15-59': 'Pop_Urb_15-19',
    'IBGE_60+': 'Pop_Urb_60+', 'Cars': 'Total_Cars', 'Motorcycles': 'Total_Motorcycles',
    'TAXES': 'Taxes', 'LAT': 'City_Latitude', 'LONG': 'City_Longitude'
})

In [179]:
data_pop_clean.head()

,City,State,Capital,Total_Pop,Total_Pop_Brazilians,Total_Pop_Foreigners,Total_Uni_Dom,Total_Uni_Dom_Urb,Total_Uni_Dom_Rur,Total_Pop_Urb,...,Pop_Urb_60+,Total_Cars,Total_Motorcycles,IDHM,IDHM_Renda,IDHM_Longevidade,IDHM_Educacao,Taxes,City_Latitude,City_Longitude
0,Abadia De Goiás,GO,0,6876.0,6876.0,0.0,2137.0,1546.0,591.0,5300.0,...,416.0,2158.0,1246.0,0.708,0.687,0.830,0.622,20554.20,-16.758812,-49.440548
1,Abadia Dos Dourados,MG,0,6704.0,6704.0,0.0,2328.0,1481.0,847.0,4154.0,...,589.0,2227.0,1142.0,0.690,0.693,0.839,0.563,12873.50,-18.487565,-47.396832
2,Abadiânia,GO,0,15757.0,15609.0,148.0,4655.0,3233.0,1422.0,10656.0,...,990.0,2838.0,1426.0,0.690,0.671,0.841,0.579,26822.58,-16.182672,-48.718812
3,Abaeté,MG,0,22690.0,22690.0,0.0,7694.0,6667.0,1027.0,18464.0,...,2681.0,6928.0,2953.0,0.698,0.720,0.848,0.556,26994.09,-19.155848,-45.446191
4,Abaetetuba,PA,0,141100.0,141040.0,60.0,31061.0,19057.0,12004.0,82956.0,...,5996.0,5277.0,25661.0,0.628,0.579,0.798,0.537,95180.48,-1.723470,-48.884404


- Remoção de valores nulos:

In [180]:
# Checando quantos valores nulos estão presentes em cada coluna:
data_pop_clean.isnull().sum()

City                     0
State                    0
Capital                  0
Total_Pop                8
Total_Pop_Brazilians     8
Total_Pop_Foreigners     8
Total_Uni_Dom           10
Total_Uni_Dom_Urb       10
Total_Uni_Dom_Rur       81
Total_Pop_Urb            8
Pop_Urb_1                8
Pop_Urb_1-4              8
Pop_Urb_5-9              8
Pop_Urb_10-14            8
Pop_Urb_15-19            8
Pop_Urb_60+              8
Total_Cars              11
Total_Motorcycles       11
IDHM                     8
IDHM_Renda               8
IDHM_Longevidade         8
IDHM_Educacao            8
Taxes                    3
City_Latitude            9
City_Longitude           9
dtype: int64

Para lidar com os valores faltantes, utilizaremos o método da biblioteca SciKit Nearest Neighbors Imputations (KNNImputer), com ele, imputaremos valores nas células faltantes baseados na média dos vizinhos mais próximos aquela tabela. Antes disso, precisamos normalizar os dados, e para isso, primeiro iremos adicionar manualmente as 9 latitudes e longitudes faltantes nos dados (pois inferir esses dados utilizando scikit não seria ideal) utilizando google Maps para coleta das coordenadas.

O processo então consiste em:

- Pesquisar e adicionar latitute e longitude das cidades faltantes, estudando caso a caso.
- Normalizar o Dataset e aplicar KNNImputer para preencher os nulos.

In [181]:
# Pesquisando quais são as latitudes e longitudes faltantes:
data_pop_clean.loc[(data_pop_clean['City_Latitude'].isnull()), ['City', 'State', 'City_Latitude', 'City_Longitude']]

,City,State,City_Latitude,City_Longitude
471,Balneário Rincão,SC,NaN,NaN
2701,Lagoa Dos Patos,RS,NaN,NaN
3116,Mojuí Dos Campos,PA,NaN,NaN
3580,Paraíso Das Águas,MS,NaN,NaN
3760,Pescaria Brava,SC,NaN,NaN
3805,Pinhal Da Serra,RS,NaN,NaN
3820,Pinto Bandeira,RS,NaN,NaN
4489,Santa Terezinha,BA,NaN,NaN
4605,São Caetano,PE,NaN,NaN


 - Checando caso a caso:

In [182]:
data_pop_clean.loc[(data_pop_clean['City_Latitude'].isnull()), :].T


,471,2701,3116,3580,3760,3805,3820,4489,4605
City,Balneário Rincão,Lagoa Dos Patos,Mojuí Dos Campos,Paraíso Das Águas,Pescaria Brava,Pinhal Da Serra,Pinto Bandeira,Santa Terezinha,São Caetano
State,SC,RS,PA,MS,SC,RS,RS,BA,PE
Capital,0,0,0,0,0,0,0,0,0
Total_Pop,NaN,NaN,NaN,NaN,NaN,2130.0,NaN,9648.0,NaN
Total_Pop_Brazilians,NaN,NaN,NaN,NaN,NaN,2130.0,NaN,9648.0,NaN
Total_Pop_Foreigners,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
Total_Uni_Dom,NaN,NaN,NaN,NaN,NaN,745.0,NaN,2891.0,NaN
Total_Uni_Dom_Urb,NaN,NaN,NaN,NaN,NaN,180.0,NaN,734.0,NaN
Total_Uni_Dom_Rur,NaN,NaN,NaN,NaN,NaN,565.0,NaN,2157.0,NaN
Total_Pop_Urb,NaN,NaN,NaN,NaN,NaN,478.0,NaN,2332.0,NaN


Com a tabela, temos registros 100% faltantes como Lagoa dos Patos no Rio Grande do Sul e uma característica semelhante a todos os registros, os dados que mais aparecem completos na tabela são o número total de carros e motos.

Devido a quantidade de dados faltantes, os registros referentes a:
- Balneário Rincão;
- Lagoa Dos Patos;
- Mojuí Dos Campos;
- Paraíso Das Águas;
- Pescaria Brava;
- Pinto Bandeira e
- São Caetano

Serão removidos. Já os registros de Pinhal Da Serra e Santa Terezinha podem ser tratados com o método da Nearest Neighbors Imputations (KNNImputer).

In [183]:
# Excluíndo registros desnecessários:
data_pop_clean = data_pop_clean.drop(data_pop_clean[data_pop_clean['Total_Pop'].isnull()].index)

In [184]:
# Checando dados que sobraram:
data_pop_clean.loc[(data_pop_clean['City_Latitude'].isnull()), :].T

,3805,4489
City,Pinhal Da Serra,Santa Terezinha
State,RS,BA
Capital,0,0
Total_Pop,2130.0,9648.0
Total_Pop_Brazilians,2130.0,9648.0
Total_Pop_Foreigners,0.0,0.0
Total_Uni_Dom,745.0,2891.0
Total_Uni_Dom_Urb,180.0,734.0
Total_Uni_Dom_Rur,565.0,2157.0
Total_Pop_Urb,478.0,2332.0


Com esses dois registros, basta inserirmos as latitudes e longitudes para poder normalizar o dataset e realizar o preenchimento dos nulos.

In [185]:
# Preenchendo dados utilziando o método .at[]:

# Pinhal da Serra:
data_pop_clean.at[3805, 'City_Latitude'] = -27.874407
data_pop_clean.at[3805, 'City_Longitude'] = -51.171822

# Santa Terezinha:
data_pop_clean.at[4489, 'City_Latitude'] = -12.770203
data_pop_clean.at[4489, 'City_Longitude'] = -39.524779

In [186]:
# Checando:
data_pop_clean[['City_Latitude', 'City_Longitude']].isnull().sum()

City_Latitude     0
City_Longitude    0
dtype: int64

Agora, iremos normalizar o dataset para podermos aplicar o método correto de lidar com os nulos. Porém para isso, iremos retirar temporariamente as variáveis categóricas, pois o método KNNImputer apenas funciona com dados escalares.

In [187]:
# Criando o Dataframe com as variáveis categóricas e de coordenadas:
var_cat = data_pop_clean[['City', 'State', 'City_Latitude', 'City_Longitude']].copy()
var_cat

,City,State,City_Latitude,City_Longitude
0,Abadia De Goiás,GO,-16.758812,-49.440548
1,Abadia Dos Dourados,MG,-18.487565,-47.396832
2,Abadiânia,GO,-16.182672,-48.718812
3,Abaeté,MG,-19.155848,-45.446191
4,Abaetetuba,PA,-1.723470,-48.884404
...,...,...,...,...
5568,Xique-Xique,BA,-10.824974,-42.725508
5569,Zabelê,PB,-8.076874,-37.093552
5570,Zacarias,SP,-21.050110,-50.055740
5571,Zé Doca,MA,-3.275481,-45.657698


In [188]:
# Dropando variáveis categóricas e de coordenadas do dataset limpo:
data_pop_clean = data_pop_clean.drop(['City', 'State', 'City_Latitude', 'City_Longitude'], axis=1)

- Normalização dos dados:

In [189]:
# Realizando a Normalização:
scaler = MinMaxScaler()
data_pop_clean = pd.DataFrame(scaler.fit_transform(data_pop_clean), columns = data_pop_clean.columns)
data_pop_clean.head()

,Capital,Total_Pop,Total_Pop_Brazilians,Total_Pop_Foreigners,Total_Uni_Dom,Total_Uni_Dom_Urb,Total_Uni_Dom_Rur,Total_Pop_Urb,Pop_Urb_1,Pop_Urb_1-4,...,Pop_Urb_10-14,Pop_Urb_15-19,Pop_Urb_60+,Total_Cars,Total_Motorcycles,IDHM,IDHM_Renda,IDHM_Longevidade,IDHM_Educacao,Taxes
0,0.0,0.000540,0.000545,0.000000,0.000531,0.000419,0.017393,0.000490,0.000533,0.000608,...,0.000644,0.000489,0.000299,0.000376,0.001095,0.653153,0.584521,0.711712,0.671521,0.000296
1,0.0,0.000524,0.000530,0.000000,0.000584,0.000400,0.024966,0.000380,0.000294,0.000392,...,0.000433,0.000370,0.000433,0.000388,0.001003,0.612613,0.596741,0.752252,0.576052,0.000231
2,0.0,0.001329,0.001330,0.001236,0.001235,0.000894,0.041975,0.001002,0.001074,0.001253,...,0.001372,0.000964,0.000743,0.000494,0.001253,0.612613,0.551935,0.761261,0.601942,0.000350
3,0.0,0.001945,0.001966,0.000000,0.002085,0.001862,0.030290,0.001748,0.001359,0.001653,...,0.001948,0.001684,0.002051,0.001206,0.002599,0.630631,0.651731,0.792793,0.564725,0.000351
4,0.0,0.012468,0.012596,0.000501,0.008619,0.005354,0.354996,0.007912,0.010459,0.010804,...,0.011348,0.007569,0.004615,0.000919,0.022614,0.472973,0.364562,0.567568,0.533981,0.000933


In [190]:
# Aplicando o médoto de KNNImputer:
imputer = KNNImputer(n_neighbors=5)
df_norm = pd.DataFrame(imputer.fit_transform(data_pop_clean),columns = data_pop_clean.columns)

In [191]:
# Visualizando dados normalizados:
df_norm

,Capital,Total_Pop,Total_Pop_Brazilians,Total_Pop_Foreigners,Total_Uni_Dom,Total_Uni_Dom_Urb,Total_Uni_Dom_Rur,Total_Pop_Urb,Pop_Urb_1,Pop_Urb_1-4,...,Pop_Urb_10-14,Pop_Urb_15-19,Pop_Urb_60+,Total_Cars,Total_Motorcycles,IDHM,IDHM_Renda,IDHM_Longevidade,IDHM_Educacao,Taxes
0,0.0,0.000540,0.000545,0.000000,0.000531,0.000419,0.017393,0.000490,0.000533,0.000608,...,0.000644,0.000489,0.000299,0.000376,0.001095,0.653153,0.584521,0.711712,0.671521,0.000296
1,0.0,0.000524,0.000530,0.000000,0.000584,0.000400,0.024966,0.000380,0.000294,0.000392,...,0.000433,0.000370,0.000433,0.000388,0.001003,0.612613,0.596741,0.752252,0.576052,0.000231
2,0.0,0.001329,0.001330,0.001236,0.001235,0.000894,0.041975,0.001002,0.001074,0.001253,...,0.001372,0.000964,0.000743,0.000494,0.001253,0.612613,0.551935,0.761261,0.601942,0.000350
3,0.0,0.001945,0.001966,0.000000,0.002085,0.001862,0.030290,0.001748,0.001359,0.001653,...,0.001948,0.001684,0.002051,0.001206,0.002599,0.630631,0.651731,0.792793,0.564725,0.000351
4,0.0,0.012468,0.012596,0.000501,0.008619,0.005354,0.354996,0.007912,0.010459,0.010804,...,0.011348,0.007569,0.004615,0.000919,0.022614,0.472973,0.364562,0.567568,0.533981,0.000933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5560,0.0,0.003975,0.004018,0.000000,0.003187,0.002368,0.093859,0.003089,0.004001,0.004493,...,0.004604,0.002731,0.002742,0.000370,0.004460,0.376126,0.331976,0.310811,0.440129,0.000235
5561,0.0,0.000113,0.000114,0.000000,0.000121,0.000121,0.005354,0.000124,0.000154,0.000204,...,0.000139,0.000111,0.000135,0.000015,0.000139,0.461712,0.340122,0.238739,0.614887,0.000126
5562,0.0,0.000136,0.000137,0.000000,0.000157,0.000163,0.004674,0.000159,0.000162,0.000192,...,0.000147,0.000152,0.000206,0.000157,0.000301,0.702703,0.600815,0.693694,0.755663,0.000138
5563,0.0,0.004387,0.004433,0.000175,0.003257,0.002128,0.126368,0.002927,0.004179,0.004586,...,0.004457,0.002556,0.002428,0.000300,0.009768,0.398649,0.323829,0.328829,0.482201,0.000314


In [192]:
# Checando nulos
df_norm.isnull().sum()

Capital                 0
Total_Pop               0
Total_Pop_Brazilians    0
Total_Pop_Foreigners    0
Total_Uni_Dom           0
Total_Uni_Dom_Urb       0
Total_Uni_Dom_Rur       0
Total_Pop_Urb           0
Pop_Urb_1               0
Pop_Urb_1-4             0
Pop_Urb_5-9             0
Pop_Urb_10-14           0
Pop_Urb_15-19           0
Pop_Urb_60+             0
Total_Cars              0
Total_Motorcycles       0
IDHM                    0
IDHM_Renda              0
IDHM_Longevidade        0
IDHM_Educacao           0
Taxes                   0
dtype: int64

In [203]:
# Juntando as variáveis categóricas e coordenadas aos dados normalizados:
df_final = var_cat.merge(df_norm, left_index=True, right_index=True)
df_final

,City,State,City_Latitude,City_Longitude,Capital,Total_Pop,Total_Pop_Brazilians,Total_Pop_Foreigners,Total_Uni_Dom,Total_Uni_Dom_Urb,...,Pop_Urb_10-14,Pop_Urb_15-19,Pop_Urb_60+,Total_Cars,Total_Motorcycles,IDHM,IDHM_Renda,IDHM_Longevidade,IDHM_Educacao,Taxes
0,Abadia De Goiás,GO,-16.758812,-49.440548,0.0,0.000540,0.000545,0.000000,0.000531,0.000419,...,0.000644,0.000489,0.000299,0.000376,0.001095,0.653153,0.584521,0.711712,0.671521,0.000296
1,Abadia Dos Dourados,MG,-18.487565,-47.396832,0.0,0.000524,0.000530,0.000000,0.000584,0.000400,...,0.000433,0.000370,0.000433,0.000388,0.001003,0.612613,0.596741,0.752252,0.576052,0.000231
2,Abadiânia,GO,-16.182672,-48.718812,0.0,0.001329,0.001330,0.001236,0.001235,0.000894,...,0.001372,0.000964,0.000743,0.000494,0.001253,0.612613,0.551935,0.761261,0.601942,0.000350
3,Abaeté,MG,-19.155848,-45.446191,0.0,0.001945,0.001966,0.000000,0.002085,0.001862,...,0.001948,0.001684,0.002051,0.001206,0.002599,0.630631,0.651731,0.792793,0.564725,0.000351
4,Abaetetuba,PA,-1.723470,-48.884404,0.0,0.012468,0.012596,0.000501,0.008619,0.005354,...,0.011348,0.007569,0.004615,0.000919,0.022614,0.472973,0.364562,0.567568,0.533981,0.000933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5560,Xambrê,PR,-23.738394,-53.486941,0.0,0.003975,0.004018,0.000000,0.003187,0.002368,...,0.004604,0.002731,0.002742,0.000370,0.004460,0.376126,0.331976,0.310811,0.440129,0.000235
5561,Xangri-Lá,RS,-29.805378,-50.039081,0.0,0.000113,0.000114,0.000000,0.000121,0.000121,...,0.000139,0.000111,0.000135,0.000015,0.000139,0.461712,0.340122,0.238739,0.614887,0.000126
5562,Xanxerê,SC,-26.874585,-52.404667,0.0,0.000136,0.000137,0.000000,0.000157,0.000163,...,0.000147,0.000152,0.000206,0.000157,0.000301,0.702703,0.600815,0.693694,0.755663,0.000138
5563,Xapuri,AC,-10.652072,-68.506295,0.0,0.004387,0.004433,0.000175,0.003257,0.002128,...,0.004457,0.002556,0.002428,0.000300,0.009768,0.398649,0.323829,0.328829,0.482201,0.000314


In [204]:
# Checando com o método describe():
df_final.describe().T

,count,mean,std,min,25%,50%,75%,max
City_Latitude,5557.0,-16.449327,8.280269,-33.687567,-22.839057,-18.091372,-8.496838,4.585440
City_Longitude,5557.0,-46.227863,6.409731,-72.916501,-50.869308,-46.515916,-41.405615,-32.435186
Capital,5557.0,0.004859,0.069541,0.000000,0.000000,0.000000,0.000000,1.000000
Total_Pop,5557.0,0.002977,0.018063,0.000000,0.000393,0.000900,0.002018,1.000000
Total_Pop_Brazilians,5557.0,0.003002,0.018111,0.000000,0.000397,0.000909,0.002029,1.000000
Total_Pop_Foreigners,5557.0,0.000648,0.015011,0.000000,0.000000,0.000000,0.000084,1.000000
Total_Uni_Dom,5557.0,0.002816,0.018124,0.000000,0.000373,0.000820,0.001814,1.000000
Total_Uni_Dom_Urb,5557.0,0.002481,0.018151,0.000000,0.000229,0.000502,0.001287,1.000000
Total_Uni_Dom_Rur,5557.0,0.043408,0.050101,0.000000,0.014435,0.027747,0.054783,1.000000
Total_Pop_Urb,5557.0,0.002623,0.017785,0.000000,0.000250,0.000572,0.001446,1.000000


In [206]:
# Checando as variáveis categóricas:
df_final.describe(exclude='number')

,City,State
count,5557,5557
unique,5285,27
top,São Domingos,MG
freq,5,853
